In [58]:
import torch
from torchmin import Minimizer

In [59]:
import numpy as np

ar = np.array([1., 2., 3., 4., 5.])

ar[2] = np.nan

np.nan_to_num(ar, nan=np.inf, copy=False)
ar

array([ 1.,  2., inf,  4.,  5.])

In [60]:
# timeit

N = 100
M = 4
v = torch.randn(N)
v
K = torch.randn(N, N).abs()

A = torch.randn(N, M)

@torch.jit.script
def f1(K,A):
    f_hat = K @ A
    f_hat_K_tch_inv = torch.linalg.solve(K, f_hat)
    return torch.trace(f_hat.T @ f_hat_K_tch_inv)

@torch.jit.script
def f2(K,A):
    f_hat = K @ A
    return torch.trace(f_hat.T @ torch.inverse(K) @ f_hat)

def f3(K,A):
    f_hat = K @ A
    f_hat_K_tch_inv = torch.linalg.solve(K, f_hat)
    return torch.trace(f_hat.T @ f_hat_K_tch_inv)

def f4(K,A):
    f_hat = K @ A
    return torch.trace(f_hat.T @ torch.inverse(K) @ f_hat)
f1(K,A)

# %timeit f1(K,A)
# %timeit f2(K,A)
# %timeit f3(K,A)
# %timeit f4(K,A)

tensor(507.1351)

In [61]:

import torch
from stpy.continuous_processes.gauss_procc import GaussianProcess
from stpy.helpers.helper import interval


# Parameters
N = 500
n = 256
d = 1
eps = 0.01
s = 1
gamma = 0.1

# Set random seed
torch.manual_seed(1)

# Generate random data
x = torch.rand(N, d).double() * 2 - 1
xtest = torch.from_numpy(interval(n, d, L_infinity_ball=1))

# True Gaussian process
GP_true = GaussianProcess(gamma=gamma, kernel_name="squared_exponential", d=d)
ytest = GP_true.sample(xtest)
GP_true.fit_gp(xtest, ytest)
y = GP_true.mean(x).clone()

# New data points
xnew = x[0, :].view(1, 1) + eps
ynew = y[0, 0].view(1, 1) + 1

# Combined data
x2 = torch.vstack([x, xnew])
y2 = torch.vstack([y, ynew])

# Initialize models
gp_cvx = GaussianProcess(gamma=gamma, kernel_name="squared_exponential", d=d, loss='huber', huber_delta=1.5)
# gp_torch = GaussianProcess(gamma=gamma, kernel_name="squared_exponential", d=d, loss='huber_torch', huber_delta=1.5)
gp_st = GaussianProcess(gamma=gamma, kernel_name="squared_exponential", d=d, loss='studentT', lam=0.005)

# gp_st.fit_gp(x2, y2)
# from torch.profiler import profile, record_function, ProfilerActivity 

# # with profile(activities=[ProfilerActivity.CPU], record_shapes=False) as prof:
# #     # Fit models
# #     gp_st.optimize_params(type="bandwidth", restarts=1, verbose=True, optimizer='pytorch-minimize', scale=1., weight=1., maxiter=1)

# # Fit models
# # %timeit gp_cvx.fit_gp(x2, y2)
# # %timeit gp_torch.fit_gp(x2, y2)

# # mean_cvx = gp_cvx.mean(xtest)
# # mean_torch = gp_torch.mean(xtest)
# mean_st = gp_st.mean(xtest)


In [ ]:
torch.manual_seed(1)
K = gp_st.kernel(x2, x2)

lam = 0.005

class StudModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(StudModel, self).__init__()
        self.linear_model = torch.nn.Linear(input_dim, output_dim, bias=False).double()

    def forward(self, x):
        output = self.linear_model(x)
        mu, v, alpha = torch.chunk(output, 3, dim=-1)
        # transform v and alpha to be positive
        v = torch.nn.functional.softplus(v) + 1. + 1e-4
        alpha = torch.nn.functional.softplus(alpha) + 1e-4
        return mu, alpha, v

def student_t_loss1(K, y, model):
    mu, v, alpha = model(K)
    v_alpha = v * alpha
    nll = torch.lgamma(0.5*v) + 0.5*torch.log(torch.pi*v_alpha) - torch.lgamma((v+1)*0.5) + ((v+1)*0.5)*torch.log((1 + (y-mu)**2 /v_alpha))
    nll = nll.sum()
    p = next(model.parameters())
    reg = lam * torch.trace(p @ K @ p.T)
    return nll + reg


model = StudModel(y2.shape[0], 3)
model2 = StudModel(y2.shape[0], 3)

model2.linear_model.weight.data = model.linear_model.weight.data.clone()

model = torch.jit.script(model, example_inputs=[K])



optimizer1 = Minimizer(model.parameters(), method="l-bfgs")

def closure1():
    optimizer1.zero_grad()
    loss = student_t_loss1(K, y2, model)
    return loss

# loss = optimizer.step(closure1)

with torch.no_grad():
    print(closure1())

%timeit closure1()


c:\Users\queim\micromambaenv\envs\stpy39\lib\site-packages\torch\jit\_script.py:1138: UserWarning: Warning: monkeytype is not installed. Please install https://github.com/Instagram/MonkeyType to enable Profile-Directed Typing in TorchScript. Refer to https://github.com/Instagram/MonkeyType/blob/master/README.rst to install MonkeyType. 
  warnings.warn(


tensor(1115.9382, dtype=torch.float64)
tensor(1115.9382, dtype=torch.float64)
5.94 ms ± 500 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.34 ms ± 651 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:

import matplotlib.pyplot as plt
 
plt.plot(xtest, mean_cvx, label='cvx')
# plt.plot(xtest, mean_torch, label='torch')
plt.plot(x, y, 'o', label='data')
plt.plot(xnew, ynew, 'o', label='new data')

plt.legend()

NameError: name 'mean_cvx' is not defined